In [ ]:
# EEX5335 Lab 3 - Memory Management Simulator
# Student: N.W.A.I.U.Karunathilaka
# Reg. No: 320259034

In [ ]:
# Parameters
PAGE_SIZE = 4096       # 4KB page size (in bytes)
VIRTUAL_ADDR_SPACE = 16 * 1024  # 16KB virtual address space (16-bit addresses)
PHYSICAL_MEM_SIZE = 8 * 1024   # 8KB physical memory

NUM_PAGES = VIRTUAL_ADDR_SPACE // PAGE_SIZE  # 4 pages
NUM_FRAMES = PHYSICAL_MEM_SIZE // PAGE_SIZE  # 2 frames
TLB_SIZE = 4           # TLB can hold 4 entries
CACHE_SIZE = 64        # Cache size in bytes
CACHE_LINE_SIZE = 16   # 16 bytes per cache line

In [ ]:
# -------------------------------
# Structures
page_table = [-1] * NUM_PAGES  # Initialize all entries to -1 (not in memory)
tlb = {}           # TLB: {page_number: frame_number}
main_memory = {}   # Physical memory: {frame_number: data}
cache = {}         # Cache: {tag: data}

In [ ]:
# -------------------------------
# Initialize main memory with dummy data
for frame in range(NUM_FRAMES):
    main_memory[frame] = [f"DATA_{frame}_{offset}" for offset in range(PAGE_SIZE)]

# Initialize page table with some mappings
page_table[0] = 0  # Virtual page 0 -> Physical frame 0
page_table[1] = 1  # Virtual page 1 -> Physical frame 1

In [ ]:
# -------------------------------
# Functions
def access_address(virtual_addr):
    """Simulate reading from a virtual address"""
    # Extract page number and offset from virtual address
    page_num = virtual_addr // PAGE_SIZE
    offset = virtual_addr % PAGE_SIZE

    print(f"\nAccessing Virtual Address: 0x{virtual_addr:04X}")
    print(f"→ Page Number: {page_num}, Offset: {offset}")

    # Step 1: Check TLB
    if page_num in tlb:
        frame = tlb[page_num]
        print("✅ TLB HIT → Frame:", frame)
    else:
        print("❌ TLB MISS")
        # Step 2: Check Page Table
        if page_num < len(page_table) and page_table[page_num] != -1:
            frame = page_table[page_num]
            print("✅ PAGE TABLE HIT → Frame:", frame)

            # Update TLB (FIFO replacement if full)
            if len(tlb) >= TLB_SIZE:
                # Remove the oldest entry
                oldest_key = next(iter(tlb))
                del tlb[oldest_key]
            tlb[page_num] = frame
        else:
            print("❌ PAGE FAULT (Page not in memory)")
            return None

    # Step 3: Calculate physical address
    physical_addr = (frame * PAGE_SIZE) + offset
    print(f"→ Physical Address: 0x{physical_addr:04X}")

    # Step 4: Check Cache (simplified)
    cache_tag = physical_addr // CACHE_LINE_SIZE

    if cache_tag in cache:
        print("✅ CACHE HIT")
        data = cache[cache_tag]
    else:
        print("❌ CACHE MISS")
        # Get data from main memory
        data = main_memory[frame][offset]

        # Update cache (simple direct mapping)
        if len(cache) >= CACHE_SIZE // CACHE_LINE_SIZE:
            # Remove a random entry (simplified)
            cache.popitem()
        cache[cache_tag] = data

    print("📦 DATA:", data)
    return data

In [ ]:
# -------------------------------
# Test simulation
if __name__ == "__main__":
    print("=== Memory Management Simulator ===")
    print(f"Page Size: {PAGE_SIZE} bytes")
    print(f"Virtual Pages: {NUM_PAGES}, Physical Frames: {NUM_FRAMES}")
    print(f"TLB Size: {TLB_SIZE} entries")
    print(f"Cache Size: {CACHE_SIZE} bytes ({CACHE_SIZE//CACHE_LINE_SIZE} lines)")

    # Test cases
    access_address(0x0000)  # TLB miss, cache miss
    access_address(0x1000)  # TLB miss, cache miss
    access_address(0x0008)  # TLB hit, cache miss
    access_address(0x1004)  # TLB hit, cache miss
    access_address(0x2000)  # Page fault (page not mapped)

=== Memory Management Simulator ===
Page Size: 4096 bytes
Virtual Pages: 4, Physical Frames: 2
TLB Size: 4 entries
Cache Size: 64 bytes (4 lines)

Accessing Virtual Address: 0x0000
→ Page Number: 0, Offset: 0
❌ TLB MISS
✅ PAGE TABLE HIT → Frame: 0
→ Physical Address: 0x0000
❌ CACHE MISS
📦 DATA: DATA_0_0

Accessing Virtual Address: 0x1000
→ Page Number: 1, Offset: 0
❌ TLB MISS
✅ PAGE TABLE HIT → Frame: 1
→ Physical Address: 0x1000
❌ CACHE MISS
📦 DATA: DATA_1_0

Accessing Virtual Address: 0x0008
→ Page Number: 0, Offset: 8
✅ TLB HIT → Frame: 0
→ Physical Address: 0x0008
✅ CACHE HIT
📦 DATA: DATA_0_0

Accessing Virtual Address: 0x1004
→ Page Number: 1, Offset: 4
✅ TLB HIT → Frame: 1
→ Physical Address: 0x1004
✅ CACHE HIT
📦 DATA: DATA_1_0

Accessing Virtual Address: 0x2000
→ Page Number: 2, Offset: 0
❌ TLB MISS
❌ PAGE FAULT (Page not in memory)
